<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 01 - SQL Server Master Instance Queries

In this tutorial you will learn how to run standard SQL Server Queries against the Master Instance (MI) in a SQL Server big data cluster. 

We'll start with a simple set of queries to explore the Instance: 



In [1]:
/* Instance Version */
SELECT @@VERSION; 
GO

/* General Configuration */
USE master;  
GO  
EXEC sp_configure;
GO

/* Databases on this Instance */
SELECT db.name AS 'Database Name'
, Physical_Name AS 'Location on Disk'
, Cast(Cast(Round(cast(mf.size as decimal) * 8.0/1024000.0,2) as decimal(18,2)) as nvarchar) 'Size (GB)'
FROM sys.master_files mf
INNER JOIN 
    sys.databases db ON db.database_id = mf.database_id
WHERE mf.type_desc = 'ROWS';
GO

SELECT * from sys.master_files


: Msg 208, Level 16, State 1, Line 3
Invalid object name 'web_customers_hdfs'.

Total execution time: 00:00:00.0061315

## Ingest data into the SQL Server Databases

Before we start working with data, we need to bring it in to the system. We have several options to do that, from the `bcp` utility to SQL Server Integration Services, the Azure Data Factory and more. 

For the structured data, we'll use the SQL Server `RESTORE` command to bring in two databases from the location we specified earlier with the `kubectl` command.

The Code below shows all of that: 

In [1]:
USE [master]
RESTORE DATABASE [WideWorldImporters] 
FROM  DISK = N'/var/opt/mssql/data/WWI.bak' 
WITH  FILE = 1
,  REPLACE
,  MOVE N'WWI_Primary' TO N'/var/opt/mssql/data/WideWorldImporters.mdf'
,  MOVE N'WWI_UserData' TO N'/var/opt/mssql/data/WideWorldImporters_UserData.ndf'
,  MOVE N'WWI_Log' TO N'/var/opt/mssql/data/WideWorldImporters.ldf'
,  MOVE N'WWI_InMemory_Data_1' TO N'/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1'
,  NOUNLOAD,  STATS = 5;
GO

USE [master]
RESTORE DATABASE [WideWorldImportersDW] 
FROM  DISK = N'/var/opt/mssql/data/WWIDW.bak' 
WITH  FILE = 1
,  REPLACE
,  MOVE N'WWI_Primary' TO N'/var/opt/mssql/data/WideWorldImportersDW.mdf'
,  MOVE N'WWI_UserData' TO N'/var/opt/mssql/data/WideWorldImportersDW_UserData.ndf'
,  MOVE N'WWI_Log' TO N'/var/opt/mssql/data/WideWorldImportersDW.ldf'
,  MOVE N'WWIDW_InMemory_Data_1' TO N'/var/opt/mssql/data/WideWorldImportersDW_InMemory_Data_1'
,  NOUNLOAD,  STATS = 5

GO

USE [master]
RESTORE DATABASE [NYC]
FROM  DISK = N'/var/opt/mssql/data/NYC.bak' 
WITH  FILE = 1
,  REPLACE
,  MOVE N'NYCTaxi_Sample' TO N'/var/opt/mssql/data/NYC.mdf'
,  MOVE N'NYCTaxi_Sample_log' TO N'/var/opt/mssql/data/NYC.ldf'
,  NOUNLOAD,  STATS = 5;
GO

USE [master]
RESTORE DATABASE [sales] 
FROM  DISK = N'/var/opt/mssql/data/sales.bak' 
WITH  FILE = 1,  NOUNLOAD,  STATS = 5

GO


## Query Data

The SQL Server 2019 big data cluster Master Instance is a SQL Server Instance - and as such has most all of the query facilities and capabilities of Microsoft SQL Server running on Linux.

**TODO:** Run some standard queries. Investigate simple ML.

In [1]:
USE WideWorldImporters;
GO

/* Show the Populations. 
Where do we have the most people?
 */
SELECT TOP 10 CityName as 'City Name'
, StateProvinceName as 'State or Province'
, sp.LatestRecordedPopulation as 'Population'
, CountryName
FROM Application.Cities AS city
JOIN Application.StateProvinces AS sp on
    city.StateProvinceID = sp.StateProvinceID
JOIN Application.Countries AS ctry on 
    sp.CountryID=ctry.CountryID


Commands completed successfully.

Total execution time: 00:00:00.0040103

(10 rows affected)

Total execution time: 00:00:00.3027652

City Name,State or Province,Population,CountryName
Addison,Michigan,9895622,United States
Adrian,Michigan,9895622,United States
Advance,Michigan,9895622,United States
Ahmeek,Michigan,9895622,United States
Akron,Michigan,9895622,United States
Alabaster,Michigan,9895622,United States
Alanson,Michigan,9895622,United States
Alba,Michigan,9895622,United States
Albion,Michigan,9895622,United States
Alden,Michigan,9895622,United States


In [2]:
/* Show Customer Sales
Where do we have the most customers?
*/
USE WideWorldImporters;
GO

SELECT TOP 10 s.CustomerID
, s.CustomerName
, sc.CustomerCategoryName
,  pp.FullName AS PrimaryContact
,  ap.FullName AS AlternateContact
,  s.PhoneNumber
,  s.FaxNumber
,  bg.BuyingGroupName
,  s.WebsiteURL
,  dm.DeliveryMethodName AS DeliveryMethod
,  c.CityName AS CityName
,  s.DeliveryLocation AS DeliveryLocation
,  s.DeliveryRun
,  s.RunPosition
FROM Sales.Customers AS s
    LEFT OUTER JOIN Sales.CustomerCategories AS sc
    ON s.CustomerCategoryID = sc.CustomerCategoryID
    LEFT OUTER JOIN [Application].People AS pp
    ON s.PrimaryContactPersonID = pp.PersonID
    LEFT OUTER JOIN [Application].People AS ap
    ON s.AlternateContactPersonID = ap.PersonID
    LEFT OUTER JOIN Sales.BuyingGroups AS bg
    ON s.BuyingGroupID = bg.BuyingGroupID
    LEFT OUTER JOIN [Application].DeliveryMethods AS dm
    ON s.DeliveryMethodID = dm.DeliveryMethodID
    LEFT OUTER JOIN [Application].Cities AS c
    ON s.DeliveryCityID = c.CityID

Commands completed successfully.

Total execution time: 00:00:00.0049281

(10 rows affected)

Total execution time: 00:00:01.9670829

CustomerID,CustomerName,CustomerCategoryName,PrimaryContact,AlternateContact,PhoneNumber,FaxNumber,BuyingGroupName,WebsiteURL,DeliveryMethod,CityName,DeliveryLocation,DeliveryRun,RunPosition
47,"Tailspin Toys (Lake Hughes, CA)",Novelty Shop,Serhat Akbulut,Nadja Johansson,(209) 555-0100,(209) 555-0101,Tailspin Toys,http://www.tailspintoys.com/LakeHughes,Delivery Van,Lake Hughes,0xE6100000010CB759619FA556414000FD63C6809C5DC0,,
46,"Tailspin Toys (Jemison, AL)",Novelty Shop,Didem ozCelik,Krista Andrejev,(205) 555-0100,(205) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Jemison,Delivery Van,Jemison,0xE6100000010CE9DC4834DC7A4040221F4F26C9AF55C0,,
45,"Tailspin Toys (Severna Park, MD)",Novelty Shop,Lap Dinh,August Astrom,(240) 555-0100,(240) 555-0101,Tailspin Toys,http://www.tailspintoys.com/SevernaPark,Delivery Van,Severna Park,0xE6100000010CF303577902894340437C163AE52253C0,,
44,"Tailspin Toys (Amanda Park, WA)",Novelty Shop,Adrijana Blagojevic,Lina Martins,(206) 555-0100,(206) 555-0101,Tailspin Toys,http://www.tailspintoys.com/AmandaPark,Delivery Van,Amanda Park,0xE6100000010C22F4D83B48BB474086B82869B4F85EC0,,
43,"Tailspin Toys (Upper Preston, WA)",Novelty Shop,Ganapati Gadiyaram,Parsa Blijleven,(206) 555-0100,(206) 555-0101,Tailspin Toys,http://www.tailspintoys.com/UpperPreston,Delivery Van,Upper Preston,0xE6100000010CE0D2D62DA7C04740FB96395D167A5EC0,,
42,"Tailspin Toys (Arietta, NY)",Novelty Shop,Chandrakanta Raut,Ivan Castellanos,(212) 555-0100,(212) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Arietta,Delivery Van,Arietta,0xE6100000010CC3F2E7DB829F4540961F5D1121A152C0,,
60,"Tailspin Toys (East Dailey, WV)",Novelty Shop,Lekha Mitra,Seeta Bose,(304) 555-0100,(304) 555-0101,Tailspin Toys,http://www.tailspintoys.com/EastDailey,Delivery Van,East Dailey,0xE6100000010C4A9E46B5E3634340273FF38304F953C0,,
41,"Tailspin Toys (Olivette, MO)",Novelty Shop,Lana Rozej,Kalpana Mukherjee,(314) 555-0100,(314) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Olivette,Delivery Van,Olivette,0xE6100000010C5C470B8629554340CE5B2B900F9856C0,,
31,"Tailspin Toys (Lake Stevens, WA)",Novelty Shop,Kimberly Pace,Shobha Gupta,(206) 555-0100,(206) 555-0101,Tailspin Toys,http://www.tailspintoys.com/LakeStevens,Delivery Van,Lake Stevens,0xE6100000010C151E34BBEE01484061376C5B14845EC0,,
57,"Tailspin Toys (Lakemore, OH)",Novelty Shop,Aile Mae,Margherita Lucchesi,(216) 555-0100,(216) 555-0101,Tailspin Toys,http://www.tailspintoys.com/Lakemore,Delivery Van,Lakemore,0xE6100000010CCB69AA82AC82444007753689E65B54C0,,


## Next Step: Data Virtualization

**TODO:** Add Link